In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForTokenClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

# Load and preprocess dataset
file_path = 'NER_Dataset.csv'
data = pd.read_csv(file_path)
data['Word'] = data['Word'].apply(eval)
data['Tag'] = data['Tag'].apply(eval)

# Extract unique tags
unique_tags = set(tag for sublist in data['Tag'] for tag in sublist)
tag_to_ix = {tag: i for i, tag in enumerate(unique_tags)}

# Prepare the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=len(tag_to_ix))

# Dataset class
class NERDataset(Dataset):
    def __init__(self, tokenizer, sentences, tags, tag_to_ix, max_len=128):
        self.sentences = sentences
        self.tags = tags
        self.tag_to_ix = tag_to_ix
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        sentence = self.sentences[idx]
        tags = self.tags[idx]

        tokens = [tokenizer.cls_token]  # Start with [CLS] token
        target_tags = [-100]  # Ignore index for special tokens
        for word, tag in zip(sentence, tags):
            word_tokens = tokenizer.tokenize(word)
            tokens.extend(word_tokens)
            target_tags.extend([tag_to_ix[tag]] + [-100] * (len(word_tokens) - 1))
        tokens.append(tokenizer.sep_token)  # End with [SEP] token
        target_tags.append(-100)

        token_ids = tokenizer.convert_tokens_to_ids(tokens)
        input_ids = token_ids + [tokenizer.pad_token_id] * (self.max_len - len(token_ids))
        attention_mask = [1] * len(token_ids) + [0] * (self.max_len - len(token_ids))
        target_tags = target_tags + [-100] * (self.max_len - len(target_tags))

        return {
            'input_ids': torch.tensor(input_ids, dtype=torch.long),
            'attention_mask': torch.tensor(attention_mask, dtype=torch.long),
            'labels': torch.tensor(target_tags, dtype=torch.long)
        }

# Prepare data loaders
sentences = list(data['Word'])
tags = list(data['Tag'])
train_sentences, test_sentences, train_tags, test_tags = train_test_split(sentences, tags, test_size=0.2, random_state=42)

train_dataset = NERDataset(tokenizer, train_sentences, train_tags, tag_to_ix)
test_dataset = NERDataset(tokenizer, test_sentences, test_tags, tag_to_ix)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Optimizer
optimizer = AdamW(model.parameters(), lr=3e-5)

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(3):
    model.train()
    total_loss = 0
    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss if hasattr(outputs, 'loss') else outputs[0]
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()

    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader)}")

# Evaluation
def evaluate_model(model, data_loader):
    model.eval()
    true_tags = []
    pred_tags = []

    with torch.no_grad():
        for batch in data_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            predictions = torch.argmax(outputs.logits, dim=2)
            labels = batch['labels']

            for i in range(labels.size(0)):
                sentence_labels = [label.item() for label in labels[i] if label != -100]
                sentence_predictions = [prediction.item() for prediction in predictions[i] if prediction != -100]

                true_tags.extend(sentence_labels)
                pred_tags.extend(sentence_predictions)

    accuracy = accuracy_score(true_tags, pred_tags)
    report = classification_report(true_tags, pred_tags, target_names=list(tag_to_ix.keys()))

    return accuracy, report

accuracy, report = evaluate_model(model, test_loader)
print(f"Accuracy: {accuracy}\n")
print(report)
